In [1]:
%config Completer.use_jedi=False

In [14]:
import torch
from torch import nn

In [18]:
VGG_types = {
    "VGG11": [64, "M", 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
    "VGG13": [64, 64, "M", 128, 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
    "VGG16": [
        64,
        64,
        "M",
        128,
        128,
        "M",
        256,
        256,
        256,
        "M",
        512,
        512,
        512,
        "M",
        512,
        512,
        512,
        "M",
    ],
    "VGG19": [
        64,
        64,
        "M",
        128,
        128,
        "M",
        256,
        256,
        256,
        256,
        "M",
        512,
        512,
        512,
        512,
        "M",
        512,
        512,
        512,
        512,
        "M",
    ],
}

#m mean maxpool num means output channel of con2d

In [34]:
class VGG_net(nn.Module):
    def __init__(self,in_channels=3,num_classes=1000):
        super(VGG_net,self).__init__()
        self.in_channels=in_channels
        self.num_classes=num_classes
        self.conv_layers=self.create_conv_layers(VGG_types['VGG16'])
        self.fc=nn.Sequential(nn.Linear(7*7*512,4096),
                              nn.ReLU(),
                              nn.Dropout(p=.5),
                              nn.Linear(4096,4096),
                              nn.ReLU(),
                              nn.Dropout(p=.5),
                              nn.Linear(4096,num_classes)
                             )
    def forward(self,x):
        x=self.conv_layers(x)
        x=x.reshape(x.shape[0],-1)
        x=self.fc(x)
        return x
    def create_conv_layers(self,architecture):
        layers=[]
        in_channels=self.in_channels
        for x in architecture:
            if type(x) ==int:
                out_channels=x
                layers+=[nn.Conv2d(in_channels=in_channels,out_channels=out_channels,
                                    kernel_size=(3,3),stride=(1,1),padding=(1,1)),
                        nn.BatchNorm2d(x),
                        nn.ReLU()]
                in_channels=x
            elif x=='M':
                layers+=[nn.MaxPool2d(stride=(2,2),kernel_size=(2,2))]
        return nn.Sequential(*layers)

In [35]:
model=VGG_net()
x=torch.randn((1,3,224,224))

In [36]:
print(model(x).shape)

torch.Size([1, 1000])
